<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%A0%9C%EC%A3%BC%EC%B9%B4%EB%93%9C%EC%82%AC%EC%9A%A9%EB%9F%89%EC%98%88%EC%B8%A1/Dacon_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# < DACON - 제주 빅데이터 경진대회(카드 사용금액 예측) >
---

<br>

### Day 1 정리
  * 데이콘에서 올려준 베이스라인 코드 따라하기
  * train/test 데이터 체크
  * 전체적 흐름 파악
  *   https://dacon.io/competitions/official/235615/codeshare/1228?page=1&dtype=recent&ptype=pub

<br>

* <b> 데이터 명세 </b> 
  * REG_YYMM : 년/월
  * CARD_SIDO_NM : 카드이용지역_시도 (가맹점 주소 기준)
  * CARD_CCG_NM : 카드이용지역_시군구 (가맹점 주소 기준)
  * STD_CLSS_NM : 업종명
  * HOM_SIDO_NM : 거주지역_시도 (고객 집주소 기준)
  * HOM_CCG_NM : 거주지역_시군구 (고객 집주소 기준)
  * AGE : 연령대 
  * SEX_CTGO_CD : 성별 (1: 남성, 2: 여성)
  * FLC : 가구생애주기 (1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구)
  * CSTMR_CNT : 이용고객수 (명)
  * AMT : 이용금액 (원)
  * CNT : 이용건수 (건)

<br>

* <b> 해결해야 하는 문제 </b>

 * 2020.04, 2020.07 기간 내 ```지역```, ```업종``` 별 ```월간 총 사용 금액``` 예측

* submission score
  * dacon base line : 7.09526 , 5위 ( 2020.06.22 )

In [1]:
## import libarary

import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv') # data load

In [2]:
## data preprocessing

data = data.fillna('') # 결측값 ''로 대체

## 날짜 처리
def grap_year(data): # year 
    data = str(data)
    return int(data[:4])

def grap_month(data): # month 
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1) # 기존 날짜컬럼 삭제

In [3]:
data.head(5)

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [4]:
## data preprocessing

df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1) # 카드이용지역_시군구 (가맹점 주소 기준), 거주지역_시군구 (고객 집주소 기준) 삭제

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False) # 지역, 업종 별 월간 총 사용 금액을 예측하기 위한 groupby

In [5]:
df.head(5)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3


In [7]:
# 인코딩
dtypes = df.dtypes # 각 컬럼별 dtype
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object': # dtype이 object인 컬럼이면  ## CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE
        # LabelEncoder로 encoding
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder

# encoding된 데이터로 변경        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [8]:
df_num.head(5) 

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,0,0,0,1,1,1,2019,1,4,311200,4
1,0,0,0,1,1,1,2019,2,3,605000,3
2,0,0,0,1,1,1,2019,6,3,139000,3
3,0,0,0,1,1,1,2019,8,3,27500,3
4,0,0,0,1,1,1,2019,9,3,395500,3


In [9]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0) ## df.sample()을 통해 sampling 단, frac=1이므로 이 경우 전체 데이터를 셔플링한 효과
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [10]:
# 훈련
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

* 예측 템플릿 부분 설명
  * 최종적으로 예측해야하는 문제는 2020.04와 2020.07의 지역별,업종별 월간 총 카드 사용량이다.
  * model fitting과정에서 학습시킨 모든 데이터로 predict하되 year,month만 바꾸어 예측한 값을 groupby 한것.

In [11]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [12]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,0,1,1,1,2020,4
1,0,0,0,1,1,1,2020,7
2,0,0,0,1,1,2,2020,4
3,0,0,0,1,1,2,2020,7
4,0,0,0,1,1,3,2020,4
...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7
1658856,16,30,14,0,2,4,2020,4
1658857,16,30,14,0,2,4,2020,7
1658858,16,30,14,0,2,5,2020,4


In [13]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [14]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [15]:
# 제출 파일 만들기
submission = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/submission/dacon_baseline.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.074107e+08
1,202004,강원,골프장 운영업,6.962323e+09
2,202004,강원,과실 및 채소 소매업,1.815306e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,9.742804e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.379857e+07


In [16]:
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.074107e+08
1,202004,강원,골프장 운영업,6.962323e+09
2,202004,강원,과실 및 채소 소매업,1.815306e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,9.742804e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.379857e+07
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.805664e+09
1390,202007,충북,한식 음식점업,2.552181e+10
1391,202007,충북,호텔업,2.060235e+08
